# California Housing Price Prediction

End-to-end regression project with preprocessing, model comparison, cross‑validation and hyperparameter tuning.


## 1. Setup

Check Python & scikit‑learn versions and import core libraries.

In [ ]:
import sys
import numpy as np
import pandas as pd

assert sys.version_info >= (3, 7)

In [ ]:
from packaging import version
import sklearn
assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
import tarfile
import urllib.request

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR

from xgboost import XGBRegressor

## 2. Load the data

We use the **California housing** dataset from Aurélien Géron's GitHub repo and cache it locally under `datasets/housing/`.

In [ ]:
def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
    with tarfile.open(tarball_path) as housing_tarball:
        housing_tarball.extractall(path="datasets")
    csv_path = Path("datasets/housing/housing.csv")
    return pd.read_csv(csv_path)

df = load_housing_data()
df.head()

## 3. Quick data inspection

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df['ocean_proximity'].value_counts()

In [ ]:
_ = df.hist(bins=50, figsize=(12, 8))
plt.tight_layout()
plt.show()

## 4. Train–test split

We first separate features and target, then create train/test sets. Splitting **before** any preprocessing avoids data leakage.

In [ ]:
X = df.drop("median_house_value", axis=1)
y = df["median_house_value"].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape

## 5. Preprocessing pipeline

We apply different transformations to numeric and categorical columns using a `ColumnTransformer` and `Pipeline`.

- **Numeric features**: median imputation + standard scaling
- **Categorical features**: most‑frequent imputation + one‑hot encoding

In [ ]:
num_attribs = ["longitude", "latitude", "housing_median_age", "total_rooms",
               "total_bedrooms", "population", "households", "median_income"]

cat_attribs = ["ocean_proximity"]

In [ ]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocessing = ColumnTransformer(
    [
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ],
    remainder="drop",
)

In [ ]:
# Sanity‑check: transformed matrices contain no NaNs
X_train_prepared = preprocessing.fit_transform(X_train)
X_test_prepared = preprocessing.transform(X_test)

print("NaNs in X_train_prepared:", np.isnan(X_train_prepared).sum())
print("NaNs in X_test_prepared:", np.isnan(X_test_prepared).sum())
print("Prepared shapes:", X_train_prepared.shape, X_test_prepared.shape)

## 6. Helper functions

Convenience functions for model evaluation and comparison.

In [ ]:
def regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, r2

In [ ]:
def cross_validate_models(models, preprocessing, X, y, cv=5):
    """Run cross‑validation for multiple models.

    Returns a DataFrame with mean CV scores.
    """
    rows = []

    for name, model in models.items():
        print(f"Running CV for: {name}")

        pipe = Pipeline([
            ("preprocessing", preprocessing),
            ("model", model),
        ])

        r2_scores = cross_val_score(
            pipe, X, y,
            cv=cv,
            scoring="r2",
            n_jobs=-1,
        )

        rmse_scores = -cross_val_score(
            pipe, X, y,
            cv=cv,
            scoring="neg_root_mean_squared_error",
            n_jobs=-1,
        )

        mae_scores = -cross_val_score(
            pipe, X, y,
            cv=cv,
            scoring="neg_mean_absolute_error",
            n_jobs=-1,
        )

        rows.append([
            name,
            r2_scores.mean(),
            rmse_scores.mean(),
            mae_scores.mean(),
            r2_scores.mean() * 100,
        ])

    results = pd.DataFrame(
        rows,
        columns=["Model", "R2_mean", "RMSE_mean", "MAE_mean", "Performance_%"],
    )
    return results.sort_values(by="R2_mean", ascending=False).reset_index(drop=True)

In [ ]:
def grid_search_all_models(models, param_grids, preprocessing, X_train, y_train, cv=3):
    """Run GridSearchCV for each model and return a summary DataFrame and dict of best estimators."""
    results = []
    best_models = {}

    for name, model in models.items():
        print(f"\nGrid search for: {name}")

        pipe = Pipeline([
            ("preprocessing", preprocessing),
            ("model", model),
        ])

        grid = GridSearchCV(
            pipe,
            param_grid=param_grids[name],
            cv=cv,
            scoring="r2",
            n_jobs=-1,
            verbose=2,
        )
        grid.fit(X_train, y_train)

        best_models[name] = grid.best_estimator_
        best_r2 = grid.best_score_
        results.append([
            name,
            best_r2,
            best_r2 * 100,
            grid.best_params_,
        ])

    summary = pd.DataFrame(
        results,
        columns=["Model", "Best_CV_R2", "Performance_%", "Best_params"],
    )
    return summary.sort_values(by="Best_CV_R2", ascending=False).reset_index(drop=True), best_models

In [ ]:
def evaluate_best_models(best_models, X_test, y_test):
    rows = []
    for name, model in best_models.items():
        print(f"Evaluating on test set: {name}")
        y_pred = model.predict(X_test)
        rmse, mae, r2 = regression_metrics(y_test, y_pred)
        rows.append([name, rmse, mae, r2, r2 * 100])
    return pd.DataFrame(
        rows,
        columns=["Model", "RMSE", "MAE", "R2", "Performance_%"],
    ).sort_values(by="R2", ascending=False).reset_index(drop=True)

## 7. Model zoo

We compare a set of tree‑based and kernel models that are well‑suited for tabular regression.

In [ ]:
models = {
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(
        objective="reg:squarederror",
        random_state=42,
    ),
    "Random Forest": RandomForestRegressor(random_state=42, n_jobs=-1),
    "Extra Trees": ExtraTreesRegressor(random_state=42, n_jobs=-1),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "SVR (RBF)": SVR(kernel="rbf"),
}

In [ ]:
param_grids = {
    "Gradient Boosting": {
        "model__n_estimators": [100, 200],
        "model__learning_rate": [0.1, 0.05],
        "model__max_depth": [3, 5],
        "model__subsample": [1.0, 0.8],
    },
    "XGBoost": {
        "model__n_estimators": [200, 400],
        "model__learning_rate": [0.1, 0.05],
        "model__max_depth": [4, 6],
        "model__subsample": [1.0, 0.8],
        "model__colsample_bytree": [1.0, 0.8],
    },
    "Random Forest": {
        "model__n_estimators": [200, 400],
        "model__max_depth": [None, 10, 20],
        "model__max_features": ["sqrt", "log2"],
        "model__min_samples_split": [2, 5],
        "model__min_samples_leaf": [1, 2],
    },
    "Extra Trees": {
        "model__n_estimators": [200, 400],
        "model__max_depth": [None, 10, 20],
        "model__max_features": ["sqrt", "log2"],
        "model__min_samples_split": [2, 5],
        "model__min_samples_leaf": [1, 2],
    },
    "Decision Tree": {
        "model__max_depth": [None, 5, 10, 20],
        "model__min_samples_split": [2, 5, 10],
        "model__min_samples_leaf": [1, 2, 4],
    },
    "SVR (RBF)": {
        "model__C": [10, 100],
        "model__gamma": ["scale", 0.1],
        "model__epsilon": [0.1, 0.01],
    },
}

## 8. Baseline cross‑validation comparison

In [ ]:
cv_results = cross_validate_models(models, preprocessing, X, y, cv=5)
cv_results

## 9. Hyperparameter tuning with Grid Search

In [ ]:
grid_results, best_models = grid_search_all_models(
    models=models,
    param_grids=param_grids,
    preprocessing=preprocessing,
    X_train=X_train,
    y_train=y_train,
    cv=3,
)
grid_results

## 10. Final evaluation on the test set

In [ ]:
test_results = evaluate_best_models(best_models, X_test, y_test)
test_results

## 11. Persist the final model

In [ ]:
import joblib

best_model_name = test_results.iloc[0]["Model"]
best_pipeline = best_models[best_model_name]
joblib.dump(best_pipeline, f"best_model_{best_model_name.replace(' ', '_').lower()}.joblib")